# EXPORTS JOBS POSTED FROM [web.byui.edu/StudentEmployment/’](web.byui.edu/StudentEmployment/) INTO A CSV FILE

In [ ]:
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

## REQUEST DATA FROM API ENDPOINT

In [ ]:
conn = http.client.HTTPSConnection("web.byui.edu")
 
conn.request("GET", "/studentemployment/api/jobs")

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

In [ ]:
data_jobs['URL'] = data_jobs.jobID.apply(lambda x: f'https://web.byui.edu/StudentEmployment/job/{x}')

In [ ]:
data_jobs['URL'].iloc[300]

In [ ]:
pd.options.display.max_columns = None
data_jobs

In [ ]:
type(data)

## DISPLAY DATA RECEIVED

In [ ]:
pd.options.display.max_columns = None

data_jobs

## SIMPLE OVERVIEW

In [ ]:
print(f'''
    THERE ARE {data_jobs.jobID.nunique()} JOBS.
    HIGHEST PAY JOB:\'{data_jobs[data_jobs.payRate==data_jobs.payRate.max()]['title'].iloc[0]}\' WITH {data_jobs.payRate.max()} DOLLARS AN HOUR.
    ONLINE JOBS: {data_jobs[data_jobs["title"].str.contains('Online')].shape[0]} OUT OF {data_jobs.shape[0]}.
''')

## CLEAN THE DATA

In [ ]:
col_dates = ['dateUpdated','startDate','endDate','beginningDate','recruitingStartDate']

data_jobs[col_dates] = data_jobs[col_dates].astype('datetime64[ns]')

data_jobs.dateUpdated = data_jobs.dateUpdated.apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

columns_to_drop = [
    'jobID', # not needed for EDA
    # 'description', # not needed for EDA
    # 'summary', # not needed for EDA
    'displayJob', # single boolean
    # 'dateUpdated',
    'startDate','endDate', # not needed for display
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    # 'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'limitApplicants', # not relevant
    'limitNumber', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]

data_jobs.drop(columns_to_drop, axis=1, inplace=True)

data_jobs['description'] = data_jobs['description'].apply(lambda x: [p.text.strip() for p in BeautifulSoup(x).find_all('p') if p.text.strip() != ''])

## OPTIONAL: Save as a CSV

In [ ]:
# data_jobs.to_csv('StudentEmployment.csv')

## FILTER OUT THE JOBS THAT HAVE 'Online','Custodian', and 'TA' IN THEIR TITLE

In [ ]:
# JOBS FILTERED. NOT ONLINE, NOR CUSTODIAN, AND NOR TA.

remove = ['Online','Custodian','TA']

data_filtered = data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False).reset_index().drop('index',axis=1)

print(f'{data_filtered.shape[0]} JOBS OUT OF {data_jobs.shape[0]} POSTED JOBS.')

pd.set_option('display.max_rows', None)

data_filtered

## OPTIONAL: Save as a CSV

In [ ]:
# data_filtered.to_csv('StudentEmployment_filtered.csv')

In [ ]:
data_jobs.payRate.value_counts().reset_index().sort_values('index')

In [ ]:
data_jobs.groupby('payRate').count()

In [ ]:
data_jobs.departmentName.value_counts()

In [ ]:
data_jobs[data_jobs.title == 'AGBUS147 OnlineTA'].iloc[0]

In [ ]:
pd.options.display.max_columns = None
data_jobs[data_jobs.jobID == 342319]

In [ ]:
data_jobs.dateUpdated.iloc[0].strftime("%Y-%m-%d")

In [ ]:
import time
ym = time.strftime("%Y-%m-%d")
ym

In [ ]:
data_jobs.dateUpdated.iloc[0]

In [ ]:
time.strftime("%Y-%m-%d")

In [ ]:
from datetime import datetime, timedelta
(datetime.today() - timedelta(1)).strftime("%Y-%m-%d") 

In [ ]:
(33/1)

In [ ]:
(1/33) - 1

In [ ]:
import requests

In [ ]:
response = requests.get('https://web.byui.edu/studentemployment/api/jobs')

In [ ]:
soup = BeautifulSoup(response.text)

In [ ]:
soup.select()

In [ ]:
import http.client
import pandas as pd

## REQUEST DATA FROM API ENDPOINT
conn = http.client.HTTPSConnection("web.byui.edu")
conn.request("GET", "/studentemployment/api/jobs")
res = conn.getresponse()
data = res.read()
info = data.decode("utf-8")
responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)
data_jobs['URL'] = data_jobs.jobID.apply(lambda x: f'https://web.byui.edu/StudentEmployment/job/{x}')

data = data_jobs

In [1]:
import requests
import pandas as pd



In [3]:
url = 'http://web.byui.edu/studentemployment/api/jobs'

response = requests.get(url)

object = response.json()

pd.DataFrame(object)

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,managerName,positionsAllocated,positionsAvailble,...,jobQuestions,isOnline,allowOnline,jobMajors,payRate,startDate,endDate,approximateHoursPerWeek,workSchedule,beginningDate
0,1588221,Hart Night Custodian,Hart Night,<p>Carpet cleaning and custodial work around c...,Cleaning campus buildings. Shift is 11:30 pm-2...,True,2022-09-26T20:02:01.6892781,David Brixey,1000.0,960.0,...,[],False,False,[],NaN,NaN,NaN,NaN,NaN,NaN
1,2221921,Grounds Crew 16,Grounds,<p>Must be on time for shifts. Work well with ...,Working in small groups to help keep the trees...,True,2022-09-26T19:46:41.3124649,Erik Kerr,1000.0,968.0,...,[],False,False,[],9.58,NaN,NaN,NaN,NaN,NaN
2,3898063,Food Services Student Jobs,University Food Services,"<p>Fast-paced, demanding job. We need dependa...",Fall Food Service Jobs Start at $10 PER HOUR a...,True,2022-09-26T18:47:01.7193712,Fauneil Schultz,200.0,149.0,...,[],False,False,[],10.00,2022-09-22T06:00:00,2022-09-27T12:00:00,10-20,Many Shifts Available,2022-09-23T06:00:00
3,341972,Biddulph/Rigby Early Morning Custodian,RIG / Biddulph FMS,<p>Carpet cleaning and custodial work around c...,Cleaning campus buildings. Shift is 4:30am-7:3...,True,2022-09-26T17:31:15.6792416,Nikki Swenson,1000.0,871.0,...,[],False,False,[],9.58,2022-09-17T00:00:00,2022-09-30T06:00:00,15,4:30AM-7:30AM Monday-Friday,2022-09-19T12:00:00
4,1738,Grounds Crew 12,Grounds,<p>Looking for students who can work Fall Seme...,Looking for students who can work Fall Semeste...,True,2022-09-26T14:51:49.6339431,Frederick Haux,1000.0,931.0,...,[],False,False,[],9.58,NaN,NaN,15-20,8:00AM-12:00PM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1358673,SOC311 OnlineTA,Campus Teaching Assistants,<p><b><span>TA is needed for the SOC 311 class...,TA is needed for the SOC 311 class. Applicants...,True,2022-09-02T20:01:39.2580835,Esther Zonts,100.0,95.0,...,[],False,False,[],10.00,2022-06-24T17:00:00,2022-09-30T14:00:00,flexible,up to 10,2022-09-12T14:00:00
83,1357823,SPED310 OnlineTA,Online Student,<p>Teaching Assistant is needed for SPED 310 o...,TA is needed for the SPED 310: Exceptional Stu...,True,2022-09-02T20:01:38.8563084,Esther Zonts,100.0,99.0,...,[],False,True,[],10.00,2022-06-24T07:00:00,2022-10-01T06:00:00,up to 10,flexible,2022-09-12T06:00:00
84,411468,ECON151 OnlineTutor,Online Student,<p><span> Online Tutor is needed for the ECON ...,Online Tutor is needed for the ECON 151: Econo...,True,2022-08-26T21:01:19.2883381,Casey McDaniel,100.0,100.0,...,[],False,True,[],9.60,2021-12-17T15:00:00,2023-01-01T02:00:00,5-10 hours (must post 7 hours of appt openings...,Hours worked may vary according to student demand,2022-04-19T01:00:00
85,32596,Chemistry Stockroom/TA/Grader,Chemistry,<p>This is a general application form for any ...,General Application form for all chemistry dep...,True,2022-06-09T20:08:58.2027174,Mike Wood,10000.0,10000.0,...,[],False,False,"[{'jobMajorId': 212, 'jobId': 32596, 'majorId'...",NaN,2022-06-09T14:00:00,NaN,NaN,Varies,NaN
